In [1]:
import spacy
import pandas as pd
import numpy as np

import Modules.solrhandler as sh
import Modules.clusterer as cls
import Modules.topicdeterminator as td
import Modules.chatbot as cb

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
#query = "anmeldung"
#target_service = 120686

In [6]:
# Import eval.csv
import os
file = os.path.realpath("../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
eval = eval.head(100)
eval.head(20)

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...
5,328200,20220131--quantEvalAllExcel.xlsx,24,Wo kann ich mich nach Corona Infektion freites...,6,True,Infektionskrankheiten melden - für Gemeinschaf...
6,330073,20220131--quantEvalAllExcel.xlsx,28,Ab wann ist man bei Johnson & Johnson geboostert,5,True,Impfung gegen Corona (SARS-CoV-2)
7,120702,20220131--quantEvalAllExcel.xlsx,36,Genesenen Bescheinigung,6,True,Meldebescheinigung beantragen
8,330150,20220131--quantEvalAllExcel.xlsx,39,"welche medikamente kann man nehmen, wenn man c...",6,True,Arzneimittel - zur Sachkundeprüfung für den Ha...
9,330174,20220131--quantEvalAllExcel.xlsx,47,ab 15.1. ist für Gäste von Restaurants Nachwe...,6,True,Gaststättengewerbe - zum Unterrichtungsnachwei...


In [7]:
#eval.iloc[2]["initialQuestion"]

Funktionen

In [8]:
def findCorrectAnswer(chatbot,targetService):

    # Find Target Cluster
    clusteredColumn = chatbot.clusterer.getClusteredColumn()
    service = chatbot.df.loc[chatbot.df["id"]==str(targetService)]
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]

    if targetCluster == chatbot.getSelectedClusterForQuestion():
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df.reset_index()
    try:
        rank = temp.index[temp["id"]==str(targetService)].values[0]
    except IndexError as e:
        return float("NaN")
    return rank + 1

Logging

In [9]:
class Logger:
    
    def __init__(self, eval_df):
        self.liste = []
        self.eval_df = eval_df
        self.chatbot = None
    
    def getDataFrame(self): # TODO
        pass
    
    def setChatbot(self, chatbot):
        self.chatbot = chatbot
    
    def log(self, ID, t, row, answer = None, initialQuery = False): 
        target_service = row["documentId"]
        
        dialogId = row["dialogId"]
        file = row["file"]
        
        service_name = row["name"]
        query = row["initialQuestion"]
        
        if initialQuery:
            nResults = len(self.chatbot.df.index)
            rank = getRank(self.chatbot,target_service)# getRank(target_service)

            row = (file, dialogId, ID, t, service_name, query, "initialQuery", None, rank, nResults)
        else:
            if answer is not None:


                nResults = len(self.chatbot.df.index)
                question = self.chatbot.generateQuestion()
                answer = answer

                rank = getRank(self.chatbot,target_service)# getRank(target_service)

                row = (file, dialogId, ID, t,  service_name, query, question, answer, rank, nResults)
            else:
                row = (file, dialogId, ID, t, service_name, query, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

Execution

In [10]:
# Initialisierung Komponenten Chatbot
solrhandler = sh.SolrHandler(1000)
clusterer = cls.Clusterer()
tpc_dterminator = td.TopicDeterminator()

In [11]:
# Experiment durchlaufen und Loggen
lg = Logger(eval_df = eval)   

for (ID, row) in eval.iterrows():
    #print(ID)
    
    query = row["initialQuestion"]
    target_service = row["documentId"]
    
    # Chatbot initialisieren
    try:
        chatbot = cb.Chatbot(query,3,solrhandler,clusterer,tpc_dterminator, forceClusters = True)
    
    # Fehler überspringen: eg. kein solr output
    except Exception as e: 
        if(str(e) == 'response'): # TODO ??
            next
        elif(str(e) == 'no solr output'):
            next
        else:
            print(e)
            
    lg.setChatbot(chatbot)
    t = 0
    answer = findCorrectAnswer(chatbot, target_service)
    lg.log(ID,t, row, answer, initialQuery = True)
    if answer is None:
        continue
    
    
    while not chatbot.isFinished():#chatbot.refineResultset(answer, recluster = True):

        # log
        t += 1
        
        answer = findCorrectAnswer(chatbot, target_service)
        chatbot.refineResultset(answer, True)
        if answer is None: 
            break
        lg.log(ID, t, row,answer)

0
[0 0 0 0 0]
0.65
1
2
3
4
5
6
7
8
9
10
11
12
13
14
'response'
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
'response'
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [12]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId", "ID","t", "name", "query", "question", "answer", "rank", "nResult"])
#log_df.head(20)

Log Analyzer

In [13]:
import Modules.log_analyzer as lo_an

In [14]:
log_analyzer=lo_an.Log_Analyzer(log_df)
delta,services,questions,means=log_analyzer.run_it()

In [15]:
log_df

,file,dialogId,ID,t,name,query,question,answer,rank,nResult,nResult_reduction,delta_nResults,delta_Rank
0,20220131--quantEvalAllExcel.xlsx,4,0,0,Änderung/Wechsel der Hauptwohnung,Ummeldung,initialQuery,None,3.0,7,NaN,NaN,NaN
1,20220131--quantEvalAllExcel.xlsx,4,0,1,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um wasserversorgung...,True,2.0,5,0.285714,2.0,1.0
2,20220131--quantEvalAllExcel.xlsx,4,0,2,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um änderung?,False,1.0,4,0.200000,1.0,1.0
3,20220131--quantEvalAllExcel.xlsx,4,0,3,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um änderung?,True,1.0,1,0.750000,3.0,0.0
4,20220131--quantEvalAllExcel.xlsx,4,1,0,Anmeldung einer Wohnung,Ummelden,initialQuery,None,NaN,7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,20220131--quantEvalAllExcel.xlsx,462,97,6,elektronischer Aufenthaltstitel (eAT) - Ausgabe,aufenthaltstitel,Geht es bei ihrem Anliegen um elektronisch?,True,1.0,1,0.982759,57.0,0.0
419,20220131--quantEvalAllExcel.xlsx,465,98,0,Personalausweis vorläufig beantragen,Mein Personalausweis ist abgelaufen,initialQuery,None,1.0,39,NaN,NaN,NaN
420,20220131--quantEvalAllExcel.xlsx,465,98,1,Personalausweis vorläufig beantragen,Mein Personalausweis ist abgelaufen,Geht es bei ihrem Anliegen um ausweisangelegen...,False,1.0,37,0.051282,2.0,0.0
421,20220131--quantEvalAllExcel.xlsx,465,98,2,Personalausweis vorläufig beantragen,Mein Personalausweis ist abgelaufen,Geht es bei ihrem Anliegen um zeitweilig?,True,1.0,1,0.972973,36.0,0.0


In [16]:
means

{'Mean Turns': 2.776470588235294,
 'Mean nResults Reduct': 0.19793642801549968,
 'Mean delta nResults': 6.913312693498452,
 'Mean delta Rank': 0.7306501547987616}

Statistics ( Alt)

In [ ]:
# avg Rank Improvement in turn t
log_df["delta_Rank"] = log_df.groupby(["ID"])["rank"].diff()
#log_df.groupby(["t"])["rank_improvement"].mean()

In [ ]:
# avg nSuggestions Improvement in turn t
log_df["delta_nResults"] = log_df.groupby(["ID"])["nResults"].diff()
#log_df.groupby(["t"])["sugg_improvement"].mean()

In [ ]:
# MRR ohne cluster refinement
#df = log_df.loc[log_df["t"] == 1]
#MRR_ohne = 1/(df["rank"].mean())
#MRR_ohne

In [ ]:
# MRR mit cluster refinement
#df = log_df.groupby(["ID"]).last("t")
#MRR_mit = 1/(df["rank"].mean())
#MRR_mit
#df.head()

In [ ]:
# avg Turns
print(log_df["t"].mean())

# bei durchschnittlich nResults
print(log_df.groupby(["ID"]).last("t")["nResults"].mean())

In [ ]:
log_df.head(20)